# How to use pytorch DataLoader
- toc: true 
- badges: true
- categories: [Pytorch]
- permalink: /how-to-use-pytorch-dataloader/

If you use pytorch as your deep learning framework, it's likely that you'll need to use [DataLoader](https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data) in your model training loop.

In this tutorial, you'll learn about
- How to construct a custom Dataset class
- How to use DataLoader to split a dataset into batches
- How to randomize a dataset in DataLoader
- How to return the randomized index in DataLoader



## What are pytorch DataLoader and Dataset
A **Dataset** is a container of your data that 
  - has a length attribute, i.e. you can do `len(dataset)`
  - is indexable, i.e.you can do `dataset[index]`


<br>

### Creating a Custom Dataset

A custom Dataset class must have three functions: 
  - `__init__` : instantiates the Dataset object
  - `__len__`: returns the number of samples in the dataset
  - `__getitem__`: loads and returns a sample from the dataset at the given index idx


Here's an example custom dataset that takes in a pandas DataFrame with columns "text" and "label".

In [ ]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.texts = df['text']
        self.labels = df['label']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        return  text,label

<br>

**Creating a DataFrame with news data**

To see how this custom dataset works, let's create a DataFrame with some news data from [agnews](https://huggingface.co/datasets/ag_news) dataset and put it into a `NewsDataset` we just created.


You can refer [here](https://www.intodeeplearning.com/how-to-load-datasets-from-hugging-face-datasets/) for a more detailed tutorial on loading datasets from the Hugging Face Datasets.

In [ ]:
#hide
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#collapse-output
from datasets import load_dataset
agnews = load_dataset('ag_news')
agnews.set_format(type="pandas")
news_df = agnews['train'][:25]

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
news_df.head()

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


**Initiating a NewsDataset**

In [ ]:
news_dataset = NewsDataset(news_df)

Check if `news_dataset` has a length

In [ ]:
len(news_dataset)

25

This means that the `news_dataset` has 25 news data.
Now check if the `news_dataset` is indexable.

In [ ]:
news_dataset[7]

('Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.',
 2)

We can see that the index 7 of `news_dataset` has a piece of news with label 2.

<br>

### Initiating a DataLoader
A **DataLoader** splits a dataset into batches which can be used for training.

You can adjust the batch size with the parameter `batch_size`.

In [ ]:
from torch.utils.data import DataLoader
news_dataloader = DataLoader(news_dataset,batch_size=4)

To load the data in batches, we can do

In [ ]:
for batch_index, (batch,label) in enumerate(news_dataloader):
  print(f'batch index: {batch_index},\n label: {label},\n batch: {batch}')


batch index: 0,
 label: tensor([2, 2, 2, 2]),
 batch: ("Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.')
batch index: 1,
 labe

<br>

## The data loading order and Sampler

A DataLoader leverages a [Sampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.Sampler) to generate indices of dataset for each batch.
We can use either the parameter `shuffle` or `sampler` to adjust the order of indices.


<br>

### SequentialSampler

By default, the parameter `shuffle` is `False`. And the DataLoader will use a [SequentialSampler](https://pytorch.org/cppdocs/api/classtorch_1_1data_1_1samplers_1_1_sequential_sampler.html) that returns indices sequentially.


In [ ]:
news_dataloader = DataLoader(news_dataset,batch_size=4)
type(news_dataloader.sampler)

torch.utils.data.sampler.SequentialSampler

We can also explicitly create a SequentialSampler and pass it into a DataLoader using the parameter `sampler`.

In [ ]:
#collapse-output
# initiate a SequentialSampler
from torch.utils.data import SequentialSampler
sequential_news_sampler = SequentialSampler(news_dataset)

# let's print out the indices in the SequentialSampler
for i in sequential_news_sampler:
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [ ]:
news_dataloader = DataLoader(news_dataset,batch_size=4,sampler = sequential_news_sampler )

for batch_index, (batch,label) in enumerate(news_dataloader):
  print(f'batch index: {batch_index},\n label: {label},\n batch: {batch}')

batch index: 0,
 label: tensor([2, 2, 2, 2]),
 batch: ("Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.')
batch index: 1,
 labe

<br>

### RandomSampler
To randomize the data loading order in `news_dataloader`, we can

**1. set the parameter `shuffle` to `True`**

In [ ]:
news_dataloader = DataLoader(news_dataset,batch_size=4,shuffle=True)
for batch_index, (batch,label) in enumerate(news_dataloader):
  print(f'batch index: {batch_index},\n label: {label},\n batch: {batch}')

batch index: 0,
 label: tensor([2, 2, 2, 2]),
 batch: ('US trade deficit swells in June The US trade deficit has exploded 19 to a record \\$55.8bn as oil costs drove imports higher, according to a latest figures.', 'Kids Rule for Back-to-School The purchasing power of kids is a big part of why the back-to-school season has become such a huge marketing phenomenon.', 'Car prices down across the board The cost of buying both new and second hand cars fell sharply over the past five years, a new survey has found.', 'Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.')
batch index: 1,
 label: tensor([2, 2, 2, 2]),
 batch: ('Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an ann

Note that the dataloader will automatically use RandomSampler as its `sampler` when `shuffle` is `True`

In [ ]:
type(news_dataloader.sampler)

torch.utils.data.sampler.RandomSampler

We can also

**2. Pass a RandomSampler to a DataLoader**

[RandomSampler](https://pytorch.org/cppdocs/api/classtorch_1_1data_1_1samplers_1_1_random_sampler.html) returns random indices.

In [ ]:
#collapse-output
# initiate a RandomSampler
from torch.utils.data import RandomSampler
random_news_sampler = RandomSampler(news_dataset)

# print out the indices in the RandomSampler
for i in random_news_sampler:
  print(i)

15
23
16
8
1
3
7
20
12
13
9
24
0
19
5
22
6
18
21
11
10
2
17
4
14


We can then pass `random_news_sampler` to the dataloader using the parameter `sampler`.

In [ ]:
random_news_dataloader = DataLoader(news_dataset,batch_size=4,sampler=random_news_sampler)

for batch_index, (batch,label) in enumerate(random_news_dataloader):
  print(f'batch index: {batch_index},\n label: {label},\n batch: {batch}')

batch index: 0,
 label: tensor([2, 2, 2, 2]),
 batch: ('Car prices down across the board The cost of buying both new and second hand cars fell sharply over the past five years, a new survey has found.', "In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market these days, but investors in value-focused mutual funds have reason to feel a bit smug -- if only because they've lost less than the folks who stuck with growth.", "Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.", 'Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insura

## Get the random indices from RandomSampler

Sometimes, we might want to get the random indices from the DataLoader.


One way to do this is to let the `__getitem__` method in Dataset return a triple (index, data, label) 

**Define a new `NewsDataset` class:**

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.texts = df['text']
        self.labels = df['label']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        return idx,text,label # <--- make the dataset return the index as well

Let's see if the NewsDataset works as desired.

In [ ]:

# Initiate a NewsDataset
news_dataset = NewsDataset(news_df)
# Access the news of index 8
news_dataset[8]

(8,
 'Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.',
 2)

You can see in the output that now the `news_dataset` also returns the index.
We can then create a DataLoader with this `news_dataset`.

In [ ]:

# initiate a news RandomSampler
random_news_sampler = RandomSampler(news_dataset)
# initiate a news DataLoader
random_news_dataloader = DataLoader(news_dataset,batch_size = 4,sampler=random_news_sampler)

When looping through the batches, use the according triple `(index, batch, label)` to store what we get form the dataloader.

In [ ]:
for batch_index, (index,batch,label) in enumerate(random_news_dataloader):
  print(f'batch index: {batch_index},\n index: {index},\n label: {label},\n batch: {batch}')


batch index: 0,
 index: tensor([ 1,  4,  7, 13]),
 label: tensor([2, 2, 2, 2]),
 batch: ('Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', 'Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.', 'Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.', "Google IPO Auction Off to Rocky Start  WASHINGTON/NEW YORK (Reuters) - The auction for Google  Inc.'s highly anticipated initial public offerin

You can see that now the batches also contain the information of indices.

---
**Reference**
- [Dataset and DataLoader begginer tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- [DataLoader](https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data)
- [SequentialSampler](https://pytorch.org/cppdocs/api/classtorch_1_1data_1_1samplers_1_1_sequential_sampler.html)
- [RandomSampler](https://pytorch.org/cppdocs/api/classtorch_1_1data_1_1samplers_1_1_random_sampler.html)

In [ ]:
#hide
- [But what are PyTorch DataLoaders really?](https://www.scottcondron.com/jupyter/visualisation/audio/2020/12/02/dataloaders-samplers-collate.html#Use-__getitem__-and-__len__)

SyntaxError: ignored